In [56]:
# Libraries required along the code
import numpy as np
np.set_printoptions(threshold = np.inf)
#from UART_TX import *

In [57]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function describing UART serial communication protocol for transmission
def UART_TX (UART_TX_CLOCK_50,UART_TX_RESET_InHigh,UART_TX_LOCK_InHigh,UART_TX_newData_InHigh,UART_TX_data_In,
             CLOCK_PER_BIT,DATAWIDTH_BUS,STATE_SIZE):
    
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    #//////////// STATES ////////////
    State_LOCKED_IDLE = np.uint8(0)
    State_UNLOCKED_IDLE = np.uint8(1)
    State_START_BIT = np.uint8(2)
    State_DATA_INIT = np.uint8(3)
    State_DATA_END = np.uint8(4)
    State_STOP_BIT = np.uint8(5)
    #//////////// SIZES ////////////
    # Ceiling of log base 2 to get the number of bits needed to store a value
    # CLOCK_PER_BIT = 50MHz/256000Bauds
    COUNTER_SIZE = int(np.log2(CLOCK_PER_BIT)) # 8 bits
    
    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    UART_TX_tx_Out = np.uint8(1)
    UART_TX_busy_Out = np.uint8(0)

    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    # Registers (Q)
    global Tx_Register,Busy_Register,State_Register,Lock_Register
    global NewData_Register,Data_Register,Counter_Register,BitCounter_Register
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # INPUT LOGIC: Combinational
    # Signals (D)
    Lock_Signal = UART_TX_LOCK_InHigh
    NewData_Signal = UART_TX_newData_InHigh
    Data_Signal = Data_Register
    State_Signal = State_Register
    Counter_Signal = Counter_Register
    BitCounter_Signal = BitCounter_Register

    if State_Register == State_LOCKED_IDLE:
        if Lock_Register:
            State_Signal = State_LOCKED_IDLE
        else:
            State_Signal = State_UNLOCKED_IDLE

    elif State_Register == State_UNLOCKED_IDLE:
        Counter_Signal = 0
        BitCounter_Signal = 0
        if NewData_Register:
            State_Signal = State_START_BIT
            Data_Signal = UART_TX_data_In
        elif not NewData_Register and not Lock_Register:
            State_Signal = State_UNLOCKED_IDLE
        else:
            State_Signal = State_LOCKED_IDLE

    elif State_Register == State_START_BIT:
        Counter_Signal = Counter_Register + 1
        if Counter_Register == CLOCK_PER_BIT-1:
            State_Signal = State_DATA_INIT
            Counter_Signal = 0
        else:
            State_Signal = State_START_BIT

    elif State_Register == State_DATA_INIT:           
        Counter_Signal = Counter_Register + 1
        if Counter_Register == CLOCK_PER_BIT-1:
            State_Signal = State_DATA_END
            Counter_Signal = 0
        else:
            State_Signal = State_DATA_INIT

    elif State_Register == State_DATA_END:   
        BitCounter_Signal = BitCounter_Register + 1
        if BitCounter_Register == DATAWIDTH_BUS-1:
            State_Signal = State_STOP_BIT
            Counter_Signal = 0
        else:
            State_Signal = State_DATA_INIT

    elif State_Register == State_STOP_BIT:
        Counter_Signal = Counter_Register + 1
        if Counter_Register == CLOCK_PER_BIT-1:
            State_Signal = State_LOCKED_IDLE
            Counter_Signal = 0
        else:
            State_Signal = State_STOP_BIT

    else:
        State_Signal = State_LOCKED_IDLE
    
    #===========================================================================
    # OUTPUTS
    #===========================================================================
    # OUTPUT LOGIC: Combinational
    # Signals (D)
    Tx_Signal = Tx_Register
    Busy_Signal = Busy_Register
    
    if State_Register == State_LOCKED_IDLE:
        Tx_Signal = np.uint8(1)
        Busy_Signal = np.uint8(1)

    elif State_Register == State_UNLOCKED_IDLE:
        Tx_Signal = np.uint8(1)
        Busy_Signal = np.uint8(0)

    elif State_Register == State_START_BIT:
        Tx_Signal = np.uint8(0)
        Busy_Signal = np.uint8(1)

    elif State_Register == State_DATA_INIT:
        Tx_Signal = Data_Register[BitCounter_Register]
        Busy_Signal = np.uint8(1)

    elif State_Register == State_DATA_END:
        Tx_Signal = Data_Register[BitCounter_Register]
        Busy_Signal = np.uint8(1)

    elif State_Register == State_STOP_BIT:
        Tx_Signal = np.uint8(1)
        Busy_Signal = np.uint8(1)

    else:
        Tx_Signal = np.uint8(1)
        Busy_Signal = np.uint8(0)
        
    # STATE REGISTER : Sequential
    if UART_TX_CLOCK_50:
        # Updating global registers
        if UART_TX_RESET_InHigh:
            Tx_Register = np.uint8(1)
            Busy_Register = np.uint8(0)
            State_Register = State_LOCKED_IDLE
            Lock_Register = np.uint8(0)
            NewData_Register = np.uint8(0)
            Data_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            Counter_Register = 0
            BitCounter_Register = 0
        else:
            Tx_Register = Tx_Signal
            Busy_Register = Busy_Signal
            State_Register = State_Signal
            Lock_Register = Lock_Signal
            NewData_Register = NewData_Signal
            Data_Register = Data_Signal
            Counter_Register = Counter_Signal
            BitCounter_Register = BitCounter_Signal

    # OUTPUT ASSIGNMENTS
    UART_TX_tx_Out = Tx_Register
    UART_TX_busy_Out = Busy_Register
    
    return UART_TX_tx_Out,UART_TX_busy_Out

In [59]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function in charge of the input and output signals (I/O)
def BB_SYSTEM (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_LOCK_InHigh,BB_SYSTEM_newData_InHigh,BB_SYSTEM_data_In):
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    # Ratio between the internal frequency and the baud rate
    # CLOCK_PER_BIT = 50MHz/256000Bauds
    FREQUENCY = 50e6
    BAUD_RATE = 115200
    CLOCK_PER_BIT = int(FREQUENCY/BAUD_RATE)
    # Data width of the imput bus
    DATAWIDTH_BUS = 8
    # Size for the states needed into the protocol
    STATE_SIZE = 3

    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    BB_SYSTEM_tx_Out = np.uint8(1)
    BB_SYSTEM_busy_Out = np.uint8(0)
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # Port map - connection between master ports and signals/registers  
    BB_SYSTEM_tx_Out,BB_SYSTEM_busy_Out = UART_TX (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_LOCK_InHigh,
                                                   BB_SYSTEM_newData_InHigh,BB_SYSTEM_data_In,
                                                   CLOCK_PER_BIT,DATAWIDTH_BUS,STATE_SIZE)
    
    # OUTPUT ASSIGNMENTS
    return BB_SYSTEM_tx_Out,BB_SYSTEM_busy_Out

In [62]:
#===========================================================================
# SIMULATION Coding
#===========================================================================
# TB_SYSTEM
# A file to build the test vector is opened and readed
file = "prueba.txt"
fileV = np.fromfile(file,dtype = 'uint8')

# The variables to treat the file are created
nBits = 8
data_In = np.zeros((nBits,np.size(fileV)),dtype = 'uint8')

# The file with integers array is converted into bits array of type unsigned integers of size 8
for i in range(np.size(fileV)):
    for n in range(nBits):
        if fileV[i] & (1 << (7 - n)):
            bit = 1
        else:
            bit = 0
        data_In[n,i] = bit

# Variable to emulate the 50MHz clock        
CLOCK_50 = 0
        
# Parameters to describe the size of the arrays
# CLOCK_PER_BIT = 50MHz/256000Bauds
FREQUENCY = 50e6
BAUD_RATE = 115200
CLOCK_PER_BIT = int(FREQUENCY/BAUD_RATE)

# The test vectors to use for the communication protocol are created
nSim = 4
resetV = np.zeros(nSim,dtype = 'uint8')
resetV[0] = np.uint8(1)
lockV = np.zeros(nSim,dtype = 'uint8')
lockV[:2] = np.uint8(1)
newDataV = np.zeros(nSim,dtype = 'uint8')
newDataV[0] = np.uint8(1)
newDataV[nSim-1] = np.uint8(1)

# Arrays composed of protocol outputs in each clock cycle
tx_OutV = np.ones((nSim,np.size(fileV),(nBits + 2)*CLOCK_PER_BIT + nBits + 2),dtype = 'uint8')
busy_OutV = np.ones((nSim,np.size(fileV),(nBits + 2)*CLOCK_PER_BIT + nBits + 2),dtype = 'uint8')
data_Out = []

# The communication protocol is tested for nSim simulations
r = 0

while r < nSim:
    s = 0
    t = 0
    
    # The communication protocol registers are defined on its initial values
    Tx_Register = np.uint8(1)
    Busy_Register = np.uint8(0)
    State_Register = np.uint8(0)
    Lock_Register = np.uint8(1)
    NewData_Register = np.uint8(0)
    Data_Register = np.zeros(nBits,dtype = 'uint8')
    Counter_Register = 0
    BitCounter_Register = 0

    while s < np.size(fileV):
        CLOCK_50 = np.uint8(0) if CLOCK_50 else np.uint8(1)
        tx_Out,busy_Out = BB_SYSTEM (CLOCK_50,resetV[r],lockV[r],newDataV[r],data_In[:,s])
        if CLOCK_50:
            tx_OutV[r,s,int(t/2)] = tx_Out
            busy_OutV[r,s,int(t/2)] = busy_Out
            data_Out.append(tx_Out)
            if busy_Out == np.uint8(0) or lockV[r]:
                #print('Out: ',tx_OutV[r,s,:])
                #print('In:  ',data_In[:,s])
                s = s + 1
                t = 0
                
            t = t + 1
    r = r + 1

# Out to the receptor Rx input
data_Out = np.array(data_Out,dtype = 'uint8')
np.savetxt('tx_Out.txt',data_Out,fmt = '%u')
print(data_Out)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 